Name: Robert S.

Algorithm idea:

1.Restrict the data we compute similarity on, i chose to create a list of potential candidates by their number of words matching with the input.
We keep just the candidates that have the maximum numbers of words matching.

Example: 
Candidates [7734: 2, 83992: 2, 902:1 ... ]
We only keep lines with the ids 7734 and 83992 that match exactly 2 words.
Using threshold=5 means adding 902 and 4 others that match less words.

Example: 

query: "baring eastern trust"

potential candidates : ["baring uk growth trust", "the r baring childrens trust", "the j baring childrens trust"]

In all 3 candidates there are 2 words matching ("baring" and "trust")

2.Use a pretrained embedding (sentence embedding or word embedding with sum/avg over words) to compute cosine similarity between the candidates and the query.

3.Sort by similarity and return the most similar one.


Additional findings and results:

1.Using various embeddings:

-Universal Sentence Encoder Multilingual (512 dim vector as output) -> ~82% on train data

-Universal Sentence Encoder Multilingual Large model (512 dim vector as output) -> 81.8% on train data

-Universal Sentence Encoder (512 dim vector as output) -> ~80% on train data

-Universal Sentence Encoder Large model (512 dim vector as output) -> ~78% on train data

-Wiki words (250 dim vector as output) -> 75.6% on train data

-NNLM-en-news (128 dim vector as output) -> 73% on train data

-Gnews-Swivel (20 dim vector as output) -> 73% on train data


2.Candidates distribution over train dataset:

(chance that the result is in the candidates , threshold = add 0/5/10/25 more candidates from the sorted list based on the number of words matchin)



-len + 0 threshold = 0.899 

![](https://drive.google.com/uc?export=view&id=1j99GUy2wgHDi1YiVSdVCHXlVYNPvtkBN)

-len + 5 threshold = 0.943

![](https://drive.google.com/uc?export=view&id=1exy7jM6eclUO4jvOT-w0wHSeRfHRF29X)

-len + 10 threshold = 0.952

![](https://drive.google.com/uc?export=view&id=1hTbxH72Mn0PKsw6QpYkcS3FlR_2LPPyy)

-len + 25 threshold = 0.967

![](https://drive.google.com/uc?export=view&id=1NxErIynVYUTu4bo7ubOq_QeEPZhi6fQK)


-Having higher threshold doesn't work that well with the similarity metric , we obtain worse results than 0 threshold.

-It would be useful to increase the threshold if the embedding is better.

-Bert/Albert/ and other language models don't work that well, the hypothesis is that the larger the embedding size the worse the results for cosine similarity (for example with bert lot of candidates have undistinguishable probabilities 0.9998, 0.9997 .. etc)

-Speed:~10-12 queries/sec on cpu

-Computing embeddings on batches from base and keeping the text with highest similarity metric takes too much time (batch=10k with Universal sentence encoder takes ~20 secs on cpu -> 20sec * 1.7mil/10k = 20*170 ~= 1h per query


-Further improvements: 
-simhash for misspelling : "bearing" and "bearings" should be bucketed in the same hash bucket with simhash or other types of hasing functions that generate same hasing bucket for small changes.
-autoencoder or VAE applied on the base and use the latent layer as embedding to compute cosine similarity



In [1]:
import numpy as np
import pandas as pd
import io
import time
import re
import sys
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics.pairwise import cosine_similarity
import torch

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [2]:
###define your data paths here
base = 'C:\\Users\\roby10\\Desktop\\French\\public_dat\\base.csv'
train  = 'C:\\Users\\roby10\\Desktop\\French\\public_dat\\train.csv'

val =   'C:\\Users\\roby10\\Desktop\\French\\public_dat\\val.csv'
valOutput = 'C:\\Users\\roby10\\Desktop\\French\\public_dat\\submission_val.csv'

test =  'C:\\Users\\roby10\\Desktop\\French\\public_dat\\test.csv'
testOutput =  'C:\\Users\\roby10\\Desktop\\French\\public_dat\\submission_test.csv'

In [3]:
def clean(text):
    text = text.lower()
    text = re.sub(r'"', '', text)
    text = re.sub(r'[-|,+éèêëàâäôöîï]', '', text)
    return text

idx = 0 
wordDict = {}
lines = []

with io.open(base, mode="r", encoding="utf-8") as f:
  header = f.readline()

  while True:
    line = f.readline().rstrip()

    if not line:
      break
    
    name, lei = line.rsplit(',', 1)
    name = clean(name)
    lines.append((name,lei))

    ###contruct a dict where words are the keys and id of the lines are the values
    ###example : 
    ###  id 1 : "barings trust"
    ###  id 2 : "barings fund"
    ###  dict {"barings": [1,2], "trust": [1], "fund":[2]}
    ###this is used for calculating the numbers of words matching
    for word in set(name.split(' ')):
      if word not in wordDict.keys():
        wordDict[word] = [idx]
      else:
        wordDict[word].append(idx)
    
    idx += 1
  
    if idx % 100000 == 0:
      print(idx)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000


In [ ]:
####Maxw = 54 words
####Maxc = 388 chacaters
#print(np.max([len(x) for (x,y) in lines]))
#print(np.max([len(x.split(' ')) for (x,y) in lines]))

In [4]:
print(str(sys.getsizeof(wordDict)/1000000) + " MB")
print(str(sys.getsizeof(lines)/1000000) + " MB")

41.943136 MB
15.6734 MB


In [5]:
def findMatch(text, threshold=0):
  text = text.lower()
  words = text.split(' ')
  candidateWords = []
  k = wordDict.keys()


  for word in words:    
    tmpCand = []

    if word in k:
      for lineID in wordDict[word]:
        tmpCand.append(lineID)
      candidateWords.append(tmpCand)
    else:
      continue
  
  ###compute occurrence probability

  allPossible = []
  for lst in candidateWords:
    allPossible.extend(lst)

  candidateProb = {}
  for candidate in allPossible:
    if candidate not in candidateProb.keys():
      candidateProb[candidate] = 1
    else:
      candidateProb[candidate] += 1

  ###TODO: double pass: get max , filter == max in case we don't use threshold
  candidateProb = dict(sorted(candidateProb.items(), key=lambda item: item[1], reverse=True))
  
  maxV = list(candidateProb.values())[0]
  filteredCand = [x for (x,y) in candidateProb.items() if y == maxV]

  ### add additional items based on threshold value
  idx = 0
  for (x,y) in candidateProb.items():
    if idx >= threshold:
      break
    if y != maxV:
      filteredCand.append(x)
      idx += 1

  candTexts = [lines[x][0] for x in filteredCand]
  filteredLeis = [lines[x][1] for x in filteredCand]

  inputEmb = embed([text])
  embeddings = embed(candTexts)

  res = cosine_similarity(inputEmb, embeddings)

  ###display the probabilities and texts based on similarity
  '''
  tmpData = pd.DataFrame()
  tmpData['name'] = candTexts
  tmpData['prob'] = res[0]
  tmpData['leis'] = filteredLeis
  tmpData.sort_values(by=['prob'], inplace=True, ascending=False)

  
  print(text)
  print(tmpData)
  '''
  
  return filteredLeis[np.argmax(res)]

In [6]:
allSamples = 0
correctSamples = 0

start = time.time()

with io.open(train, mode="r", encoding="utf-8") as f:
  header = f.readline()

  while True:
    line = f.readline().rstrip()

    if not line:
      break
   
    name, lei = line.rsplit(',', 1)
    predLei = findMatch(name)

    if predLei == lei:
      correctSamples += 1

    allSamples += 1

    ####debug purpose
    #if allSamples > 2:
    #  assert 0

duration = time.time() - start
print("Avg results: " + str(float(correctSamples/allSamples)) + " " + str(allSamples) + " samples took " + str(duration) + "secs     speed: " + str(allSamples/duration) + " samples/sec" )

Avg results: 0.8015122873345936 529 samples took 43.45313549041748secs     speed: 12.17403517674019 samples/sec


In [7]:
#####predict on validation data
allSamples = 0

fout = io.open(valOutput, mode="w", encoding="utf-8")
fout.write("lei\n")
with io.open(val, mode="r", encoding="utf-8") as f:
  header = f.readline()

  while True:
    line = f.readline().rstrip()

    if not line:
      break

    predLei = findMatch(line)

    fout.write(predLei + "\n")
    allSamples += 1
fout.close()

In [8]:
#####predict on test data
allSamples = 0

fout = io.open(testOutput, mode="w", encoding="utf-8")
fout.write("lei\n")
with io.open(test, mode="r", encoding="utf-8") as f:
  header = f.readline()

  while True:
    line = f.readline().rstrip()

    if not line:
      break

    predLei = findMatch(line)
    fout.write(predLei + "\n")
    allSamples += 1
fout.close()